In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re

import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

from langdetect import detect

%matplotlib inline

In [2]:
df = pd.read_csv('data/online_ship_listing_data.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.drop(['Unnamed: 0', 'hull_shape'], axis = 1, inplace = True)

In [ ]:
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
df.describe()

# Outliers

In [ ]:
df.price.hist()
plt.show();

In [ ]:
df['before_2000'] = df['year'] < 2000
avg_prices = df.groupby('before_2000')['price'].mean().reset_index()

sns.barplot(x='before_2000', y='price', data=avg_prices)
plt.xlabel('Before 2000')
plt.ylabel('Average Price')
plt.title('Average price before and after the year of 2000')
plt.xticks([0, 1], ['After 2000', 'Before 2000'])
plt.show();

In [ ]:
df.length.hist()
plt.show();

In [ ]:
sns.scatterplot(x=df['length'], y=df['price'], data = df)
plt.title('Scatter plot of length vs price of ship')
plt.show();

In [ ]:
df[df['price'] == df.price.max()]

In [ ]:
df[df['length'] == df.length.max()]

There are outliers in price and length

In [ ]:
def clip_outliers(df,col):
    q1 = df[col].quantile(0.25)
    q3 = df[col].quantile(0.75)
    upper_threshold = q3 + (1.5 * (q3 - q1))
    df[col] = df[col].clip(upper=upper_threshold)

In [ ]:
for col in ['price', 'length']:
    clip_outliers(df, col)

In [ ]:
df.describe()

In [ ]:
df.price.hist()
plt.show();

In [ ]:
df['before_2000'] = df['year'] < 2000
avg_prices = df.groupby('before_2000')['price'].mean().reset_index()

sns.barplot(x='before_2000', y='price', data=avg_prices)
plt.xlabel('Before 2000')
plt.ylabel('Average Price')
plt.title('Average price before and after the year of 2000')
plt.xticks([0, 1], ['After 2000', 'Before 2000'])
plt.show();

In [ ]:
df.length.hist()
plt.show()

In [ ]:
sns.scatterplot(x=df['length'], y=df['price'], data = df)
plt.title('Scatter plot of length vs price of ship')
plt.show();

# EDA

1. Imputing missing values for fuel_type and model
2. Avg Price by Country
3. Avg Price by Manufacturer
4. Avg Price by Fuel Type
5. Avg Price by Category

## Imputing missing values for fuel_type and model

In [ ]:
df.info()

In [ ]:
def impute_knn(df, subset, text_columns):
    #divide data into 2
    data_complete = df.dropna(subset=subset).reset_index(drop = True)
    data_missing = df[df[subset].isnull()].reset_index(drop = True)
    #data_missing[subset] = data_missing.astype('str')
    
    #create tfidf vectorizer, join all the text in the text_columns into one string
    vectorizer = TfidfVectorizer()
    text_data = data_complete[text_columns].astype('str').apply(lambda x: ' '.join(x), axis=1)
    #create tfidf_matrix
    tfidf_matrix = vectorizer.fit_transform(text_data)
    
    #fit the model
    nn_model = NearestNeighbors(n_neighbors=3)  # Choose an appropriate value
    nn_model.fit(tfidf_matrix)
    for idx, row in data_missing.iterrows():
        #convert text_columns 
        row[text_columns] = row[text_columns].astype('str')
        #transform the text_columns in the data_missing into vector representation
        text_representation = vectorizer.transform([' '.join(row[text_columns])])
        #distance calculation
        _, indices = nn_model.kneighbors(text_representation)
        #get indices from the current row from data_complete
        neighbor_models = data_complete.loc[indices[0], subset]
        #calculate most frequent value
        imputed_data = neighbor_models.mode()[0]
        #impute the data
        data_missing.loc[idx, subset] = imputed_data
    
    data_complete = pd.DataFrame(np.concatenate((data_complete, data_missing), axis=0), columns = df.columns)
    return data_complete

In [ ]:
df_model = impute_knn(df, 'model', ['country', 'manufacturer', 'offerer', 'category', 'hull_material'])

In [ ]:
df_fuel = impute_knn(df_model, 'fuel_type', ['country', 'manufacturer', 'offerer', 'category', 'hull_material', 'model'])

In [ ]:
df_raw = df.copy()
df = df_fuel

## Exploring price and its correlations with other variables

In [ ]:
df.head()

In [ ]:
df.price.describe()

In [ ]:
df.country.value_counts()

In [ ]:
top_countries = df['country'].value_counts().head(10).index

In [ ]:
df['new_country'] = df['country'].apply(lambda x:x if x in top_countries else 'other')

In [ ]:
df.new_country.value_counts()

In [ ]:
price_by_country = df.groupby('new_country')['price'].mean().reset_index()

sns.barplot(x='new_country', y='price', data=price_by_country)
plt.xlabel('Country')
plt.ylabel('Average Price')
plt.title('Average price by country')
plt.xticks(rotation=45)
plt.show();

In [ ]:
price_by_hull_material = df.groupby('hull_material')['price'].mean().reset_index()

sns.barplot(x='hull_material', y='price', data=price_by_hull_material)
plt.xlabel('Hull Material')
plt.ylabel('Average Price')
plt.title('Average price by hull material')
plt.xticks(rotation=90)
plt.show();

In [ ]:
df[df['fuel_type'].isnull()]

In [ ]:
#fuel type
df.dropna(subset='fuel_type', inplace = True) #drop 

In [ ]:
price_by_fuel_type = df.groupby('fuel_type')['price'].mean().reset_index()

sns.barplot(x='fuel_type', y='price', data=price_by_fuel_type)
plt.xlabel('Hull Material')
plt.ylabel('Average Price')
plt.title('Average price by fuel type')
plt.xticks(rotation=45)
plt.show();

In [ ]:
price_by_category = df.groupby('category')['price'].mean().reset_index()

sns.barplot(x='category', y='price', data=price_by_category)
plt.xlabel('Category')
plt.ylabel('Average Price')
plt.title('Average price by category')
plt.xticks(rotation=90)
plt.show();

# Text Data

In [ ]:
#word count
df['word_count'] = df['description'].apply(lambda x: len(re.findall(r'\w+', x)))

In [ ]:
df['word_count'].mean()

In [ ]:
df['word_count'].describe()

In [ ]:
df = df[df['word_count'] > 20]

In [ ]:
df.shape

In [ ]:
df[df['word_count'] == df['word_count'].min()].description

In [ ]:
df['description_language'] = df['description'].apply(lambda x:detect(x))

In [ ]:
df['description_language'].value_counts()

In [ ]:
df = df[df['description_language'] == 'en']

In [ ]:
df['description'] = df['description'].str.replace('show more', '')

In [ ]:
df.head()

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
import re
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Preprocessing the text data

REPLACE_BY_SPACE_RE = re.compile("[/(){}\[\]\|@,;!]")
BAD_SYMBOLS_RE = re.compile("[^0-9a-z #+_]")
STOPWORDS_nlp = set(stopwords.words('english'))

#Custom Stoplist
stoplist = ["i","project","living","home",'apartment',"pune","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd","your",
            "yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers","herself","it",
            "it's","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this","that","that'll",
            "these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did",
            "doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about",
            "against","between","into","through","during","before","after","above","below","to","from","up","down","in","out",
            "on","off","over","under","again","further","then","once","here","there","when","where","why","all","any",
            "both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too",
            "very","s","t","can","will","just","don","don't","should","should've","now","d","ll","m","o","re","ve","y","ain",
            "aren","couldn","didn","doesn","hadn","hasn",
            "haven","isn","ma","mightn","mustn","needn","shan","shan't",
            "shouldn","wasn","weren","won","rt","rt","qt","for",
            "the","with","in","of","and","its","it","this","i","have","has","would","could","you","a","an",
            "be","am","can","edushopper","will","to","on","is","by","ive","im","your","we","are","at","as","any","ebay","thank","hello","know",
            "need","want","look","hi","sorry","http", "https","body","dear","hello","hi","thanks","sir","tomorrow","sent","send","see","there","welcome","what","well","us"]

STOPWORDS_nlp.update(stoplist)

# Function to preprocess the text
def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.replace("\d+"," ") # removing digits
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #removing mentions and urls
    text = text.lower() # lowercase text
    text =  re.sub('[0-9]+', '', text)
    text = REPLACE_BY_SPACE_RE.sub(" ", text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub(" ", text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS_nlp]) # delete stopwors from text
    text = text.strip()
    return text


In [ ]:
df_text = df['description'].apply(text_prepare)

In [ ]:
df_text.head()

In [ ]:
text = " ".join(sent for sent in df_text)
print("There are {} words in the text.".format(len(text)))

In [ ]:
wordcloud = WordCloud(background_color = "white", height=1200, width= 1600, collocations=True , max_words= 100).generate(text)
plt.figure(figsize= (16,12))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
wordcloud = WordCloud(background_color = "white", height=1200, width= 1600, collocations=False , max_words= 100).generate(text)
plt.figure(figsize= (16,12))
plt.imshow(wordcloud, interpolation = 'bilinear')
plt.axis("off")
plt.show()

In [ ]:
df['description_cleaned'] = df['description'].astype('str').apply(text_prepare)

In [ ]:
from collections import Counter

def pos_counter(x,pos):
    """
    Returns the count for the given parts of speech tag
    
    NN - Noun
    VB - Verb
    JJ - Adjective
    RB - Adverb
    """
    tokens = nltk.word_tokenize(x.lower())
    tokens = [word for word in tokens if word not in STOPWORDS_nlp]
    text = nltk.Text(tokens)
    tags = nltk.pos_tag(text)
    counts = Counter(tag for word,tag in tags)
    return counts[pos]

df['noun_counts'] = df['description_cleaned'].apply(lambda x: pos_counter(x,'NN'))
df['verb_counts'] = df['description_cleaned'].apply(lambda x: (pos_counter(x,'VB')+pos_counter(x,'RB')))
df['adjective_counts'] = df['description_cleaned'].apply(lambda x: pos_counter(x,'JJ'))

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
description_vectorizer = TfidfVectorizer(ngram_range=(2, 2), max_features = 10)

In [ ]:
tfidf_object = description_vectorizer.fit(df['description_cleaned'])

In [ ]:
import pickle
fileName = 'model/tfidf_vectorizer.pkl'
with open(fileName,'wb') as f:
    pickle.dump(tfidf_object,f)

In [ ]:
tfidf_matrix = tfidf_object.transform(df['description_cleaned'])

In [ ]:
tfidf_matrix.shape

In [ ]:
feature_names = description_vectorizer.get_feature_names_out()

In [ ]:
tfidf_object.get_feature_names_out()

In [ ]:
tfidf_array = tfidf_matrix.toarray()

In [ ]:
tfidf_df = pd.DataFrame(tfidf_array, columns=feature_names)

In [ ]:
tfidf_df.shape

In [ ]:
df.shape

In [ ]:
df_use = pd.concat([df.reset_index(drop=True),tfidf_df.reset_index(drop=True)],axis=1)

In [ ]:
df_use.head()

## Feature engineering - price-based feature

In [ ]:
price_by_hull_material = df.groupby('hull_material')['price'].mean().to_dict()

import pickle
fileName = 'model/price_by_hull_material.pkl'
with open(fileName,'wb') as f:
    pickle.dump(price_by_hull_material,f)

average_value_hull_material = sum(price_by_hull_material.values()) / len(price_by_hull_material)
df['avg_price_by_hull_material'] = df['hull_material'].apply(lambda x: price_by_hull_material.get(x, average_value_hull_material))

In [ ]:
price_by_hull_material

In [ ]:
price_by_fuel_type = df.groupby('fuel_type')['price'].mean().to_dict()

import pickle
fileName = 'model/price_by_fuel_type.pkl'
with open(fileName,'wb') as f:
    pickle.dump(price_by_fuel_type,f)

average_value_fuel_type = sum(price_by_fuel_type.values()) / len(price_by_fuel_type)
df['avg_price_by_fuel_type'] = df['fuel_type'].apply(lambda x: price_by_fuel_type.get(x, average_value_fuel_type))

In [ ]:
price_by_fuel_type

In [ ]:
price_by_category = df.groupby('category')['price'].mean().to_dict()

import pickle
fileName = 'model/price_by_category.pkl'
with open(fileName,'wb') as f:
    pickle.dump(price_by_category,f)

average_value_category = sum(price_by_category.values()) / len(price_by_category)
df['avg_price_by_category'] = df['category'].apply(lambda x: price_by_category.get(x, average_value_category))

In [ ]:
price_by_category

In [ ]:
df.head()

In [ ]:
df = pd.get_dummies(df, columns=['category'], prefix='category', drop_first=True)
df = pd.get_dummies(df, columns=['fuel_type'], prefix='fuel', drop_first=True)
df = pd.get_dummies(df, columns=['hull_material'], prefix='hull', drop_first=True)

In [ ]:
column_names = df.columns

In [ ]:
column_names

In [ ]:
filtered_columns = [col for col in column_names if col.startswith('category')]
for col in filtered_columns:
    df[col] = df[col].astype('float')

In [ ]:
df.info()

In [ ]:
df.model.value_counts()

In [ ]:
df_test = pd.read_csv('data/online_ship_listing_data.csv')

In [ ]:
df_test.year.value_counts()

In [ ]:
df.category.value_counts()

In [ ]:
df.model.value_counts()

In [4]:
def has_non_ascii(s):
    for char in s:
        if ord(char) > 127:
            return True
    return False

In [5]:
df = df.applymap(lambda x: x if not isinstance(x, str) or not has_non_ascii(x) else x.encode('ascii', 'ignore').decode('ascii'))

In [33]:
def clean_columns(column_list):
    all_cols = column_list
    
    modified_list = []

    for item in all_cols:
        item = str(item).lower()
        modified_item = re.sub(r'[^a-zA-Z0-9]', '_', item)
        modified_list.append(modified_item)
    
    final_list = []
    
    for i in modified_list:
        cleaned_column_name = re.sub(r'_+', '_', i)
        final_list.append(cleaned_column_name)
    
    final_list = [col.strip('_') for col in final_list]
        
    return final_list

In [25]:
df['fuel_type'] = clean_columns(df['fuel_type'].tolist())

In [26]:
df['fuel_type'].value_counts()

fuel_type
diesel        1560
nan            115
petrol          98
electrical      39
other           38
Name: count, dtype: int64

In [30]:
df['hull_material'] = clean_columns(df['hull_material'].tolist())

In [31]:
df.hull_material.value_counts()

hull_material
gfk_fiberglas_polyester    843
other                      535
wood                       142
composite                  142
steel                      114
aluminium                   66
pvc                          5
ferrocement                  3
Name: count, dtype: int64

In [32]:
clean_columns(df['model'].tolist())

['baarda_zeegrundel',
 '2800',
 'classic_schooner',
 'van_der_graaf_bv_schooner',
 'gulet',
 'custom',
 'dsc_passenger_catamaran',
 'sons_38meter',
 'eldorado',
 'aegean_builders_sa_poseidon_123',
 'gulet',
 'gulet',
 'motorsailer',
 'motorsailer_ketch',
 'nan',
 'motorsailer',
 'ketch',
 'mirror_yacht_shipyard_built_35_meter_ketch_motorsailer',
 'van_de_stadt_110',
 'baltimore_fishing',
 '32_4_m_ketch',
 'gulet',
 '34m',
 'maxi_catamaran',
 'cata_multiplast_110',
 'ocean_pearl_ex_club_med',
 'gulet',
 'sailing_yacht',
 'gulet',
 'gulet_103_ft',
 'gulet',
 'cuno_marine_31',
 '3100',
 'gulet',
 'cantiere_craglietto',
 'ketch',
 '25s',
 '93_savarona',
 'ketch',
 'besanewer_traditionssegler',
 'dubois_92',
 '91',
 'paviljoentjalk',
 '27m_ketch',
 'catamarano_jamadhar',
 '27m_ketch',
 'aynakc_26_80_m',
 'classic_wishbone_ketch',
 'gulet',
 'garcia_85',
 'gulet_ketch',
 'jfa_vitters_82',
 'carbon_ocean_82',
 '60',
 '73',
 '25_meter_tweemaster',
 'gulet_aynakic',
 'ne_demande_pas',
 'bermuda

In [34]:
df['model'] = clean_columns(df['model'].tolist())

In [35]:
df.to_excel('test_model.xlsx', index = False)

In [37]:
df.model.unique().tolist()

['baarda_zeegrundel',
 '2800',
 'classic_schooner',
 'van_der_graaf_bv_schooner',
 'gulet',
 'custom',
 'dsc_passenger_catamaran',
 'sons_38meter',
 'eldorado',
 'aegean_builders_sa_poseidon_123',
 'motorsailer',
 'motorsailer_ketch',
 'nan',
 'ketch',
 'mirror_yacht_shipyard_built_35_meter_ketch_motorsailer',
 'van_de_stadt_110',
 'baltimore_fishing',
 '32_4_m_ketch',
 '34m',
 'maxi_catamaran',
 'cata_multiplast_110',
 'ocean_pearl_ex_club_med',
 'sailing_yacht',
 'gulet_103_ft',
 'cuno_marine_31',
 '3100',
 'cantiere_craglietto',
 '25s',
 '93_savarona',
 'besanewer_traditionssegler',
 'dubois_92',
 '91',
 'paviljoentjalk',
 '27m_ketch',
 'catamarano_jamadhar',
 'aynakc_26_80_m',
 'classic_wishbone_ketch',
 'garcia_85',
 'gulet_ketch',
 'jfa_vitters_82',
 'carbon_ocean_82',
 '60',
 '73',
 '25_meter_tweemaster',
 'gulet_aynakic',
 'ne_demande_pas',
 'bermudan_ketch_95',
 'r_p_79_maxi',
 'truly_classic_78',
 'vplp_77',
 '77',
 'ketch_trehard_24m',
 'lady_rose',
 'caique_turque',
 'sw78'

In [22]:
clean_columns(df['category'].tolist())

['lastkahnsegelboote',
 'segelboote mit kajte',
 'slup boote',
 'schoner',
 'segelyachten',
 'sonstige (segelboote)',
 'katamarane und trimarane',
 'klassische segelboote',
 'slup boote',
 'klassische segelboote',
 'racer/ cruiser segelboote',
 'decksalon segelboote',
 'motorsegler (segel)',
 'segelyachten',
 'kommerzielle schiffe (segel)',
 'motorsegler (segel)',
 'gulets (segel)',
 'motorsegler (segel)',
 'decksalon segelboote',
 'slup boote',
 'ketsch segelboote',
 'segelyachten',
 'gulets (segel)',
 'segelyachten',
 'daysailers',
 'katamarane und trimarane',
 'gulets (segel)',
 'slup boote',
 'gulets (segel)',
 'gulets (segel)',
 'motorsegler (segel)',
 'katamarane',
 'slup boote',
 'gulets (segel)',
 'ketsch segelboote',
 'slup boote',
 'segelyachten',
 'segelyachten',
 'gulets (segel)',
 'klassische segelboote',
 'ketsch segelboote',
 'slup boote',
 'slup boote',
 'motorsegler (segel)',
 'katamarane',
 'segelyachten',
 'gulets (segel)',
 'segelyachten',
 'gulets (segel)',
 'slup 